<a href="https://colab.research.google.com/github/ThomasL642/AI-Boss/blob/main/AI_Boss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Import Packages**

In [ ]:
import numpy as np
import random
import keras
import tensorflow
import matplotlib.pyplot as plt
from keras.layers import Activation, Dense
from keras import Sequential
from collections import deque
from keras.optimizers import Adam
from keras.activations import relu, linear

#**Enviroment**

In [ ]:
class company_env():

    def __init__(self):
        #   BOSS
        self.action_space_boss = np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21], dtype=np.int64)
        self.observation_space_boss = np.arange(start = -100000, stop = 100000, dtype=np.int64)
        self.state_boss = 0

        #   WORKER 0
        #   Skill:                      1
        #   Endurance Level:            1
        #   Money Motivation Level:     0.9
        #
        self.action_space_worker_0 = np.array([0,1], dtype=np.int64)
        self.observation_space_worker_0 = np.arange(start = -100000, stop = 100000, dtype=np.int64)
        self.state_worker_0 = 0 
        self.worker_0_skill = 1                    
        self.worker_0_endurance_level = 1           
        self.worker_0_money_motivation_level = 0.9

        #   WORKER 1
        #   Skill:                      5
        #   Endurance Level:            1
        #   Money Motivation Level:     0.7
        #
        self.action_space_worker_1 = np.array([0,1], dtype=np.int64)
        self.observation_space_worker_1 = np.arange(start = -100000, stop = 100000, dtype=np.int64)
        self.state_worker_1 = 0 
        self.worker_1_skill = 5 
        self.worker_1_endurance_level = 1
        self.worker_1_money_motivation_level = 0.7

        #   WORKER 2
        #   Skill:                      1
        #   Endurance Level:            5
        #   Money Motivation Level:     0.5
        #
        self.action_space_worker_2 = np.array([0,1], dtype=np.int64)
        self.observation_space_worker_2 = np.arange(start = -100000, stop = 100000, dtype=np.int64)
        self.state_worker_1 = 0 
        self.worker_2_skill = 1 
        self.worker_2_endurance_level = 5
        self.worker_2_money_motivation_level = 0.5

        #   WORKER 3
        #   Skill:                      5
        #   Endurance Level:            5
        #   Money Motivation Level:     0.3
        #
        self.action_space_worker_3 = np.array([0,1], dtype=np.int64)
        self.observation_space_worker_3 = np.arange(start = -100000, stop = 100000, dtype=np.int64)
        self.state_worker_3 = 0 
        self.worker_3_skill = 5
        self.worker_3_endurance_level = 5
        self.worker_3_money_motivation_level = 0.3

        self.total_days = 365 #repesenting 1 year
        
    def boss_step(self, action_boss):

        #actions
        self.take_action_boss(action_boss, self.worker_0_pay, self.worker_1_pay, self.worker_2_pay, self.worker_3_pay, self.company_money)

        #Day off for worker 0
        if action_boss != 0:
            self.worker_0_money += self.worker_0_pay
            self.company_money -= self.worker_0_pay
            self.worker_0_day_on += 1
            self.worker_0_endurance = ((0.4641588837 * (1.165914401) ** self.worker_0_day_on) * (self.worker_0_endurance_level))
            self.worker_0_pay_motivation = (self.worker_0_pay * self.worker_0_money_motivation_level)
        else:
            self.worker_0_day_on -= 1
            self.worker_0_endurance = ((0.4641588837 * (1.165914401) ** self.worker_0_day_on) * (self.worker_0_endurance_level))
            self.worker_0_pay_motivation = (self.worker_0_pay * self.worker_0_money_motivation_level) 

        #Day off for worker 1
        if action_boss != 1:
            self.worker_1_money += self.worker_1_pay
            self.company_money -= self.worker_1_pay
            self.worker_1_day_on += 1
            self.worker_1_endurance = ((0.4641588837 * (1.165914401) ** self.worker_1_day_on) * (self.worker_1_endurance_level))
            self.worker_1_pay_motivation = (self.worker_1_pay * self.worker_1_money_motivation_level)
        else:
            self.worker_1_day_on -= 1
            self.worker_1_endurance = ((0.4641588837 * (1.165914401) ** self.worker_1_day_on) * (self.worker_1_endurance_level))
            self.worker_1_pay_motivation = (self.worker_1_pay * self.worker_1_money_motivation_level)

        #Day off for worker 2
        if action_boss != 2:
            self.worker_2_money += self.worker_2_pay
            self.company_money -= self.worker_2_pay 
            self.worker_2_day_on += 1
            self.worker_2_endurance = ((0.4641588837 * (1.165914401) ** self.worker_2_day_on) * (self.worker_2_endurance_level))
            self.worker_2_pay_motivation = (self.worker_2_pay * self.worker_2_money_motivation_level)
        else:
            self.worker_2_day_on -= 1
            self.worker_2_endurance = ((0.4641588837 * (1.165914401) ** self.worker_2_day_on) * (self.worker_2_endurance_level)) 
            self.worker_2_pay_motivation = (self.worker_2_pay * self.worker_2_money_motivation_level) 

        #Day off for worker 3
        if action_boss != 3:
            self.worker_3_money += self.worker_3_pay
            self.company_money -= self.worker_3_pay 
            self.worker_3_day_on += 1
            self.worker_3_endurance = ((0.4641588837 * (1.165914401) ** self.worker_3_day_on) * (self.worker_3_endurance_level))
            self.worker_3_pay_motivation = (self.worker_3_pay * self.worker_3_money_motivation_level)
        else:
            self.worker_3_day_on -= 1
            self.worker_3_endurance = ((0.4641588837 * (1.165914401) ** self.worker_3_day_on) * (self.worker_3_endurance_level))

            self.worker_3_pay_motivation = (self.worker_3_pay * self.worker_3_money_motivation_level)

        #reward
        self.boss_reward = self.company_money
        self.state_boss = self.company_money

        self.worker_0_reward = self.worker_0_money
        self.state_worker_0 = self.worker_0_money

        self.worker_1_reward = self.worker_1_money
        self.state_worker_1 = self.worker_1_money
        
        self.worker_2_reward = self.worker_2_money
        self.state_worker_2 = self.worker_2_money

        self.worker_3_reward = self.worker_3_money
        self.state_worker_3 = self.worker_3_money

        #reduce time
        self.total_days -= 1

        #is done?
        if self.total_days <= 0:
            done = True
        else:
            done = False

        info_boss = {}
        info_worker_0 = {}
        info_worker_1 = {}
        info_worker_2 = {}
        info_worker_3 = {}

        return self.worker_0_pay_motivation, self.worker_1_pay_motivation, self.worker_2_pay_motivation, self.worker_3_pay_motivation, self.state_worker_0, self.state_worker_1, self.state_worker_2, self.state_worker_3, self.state_boss, self.boss_reward, self.worker_0_reward, self.worker_1_reward, self.worker_2_reward, self.worker_3_reward, done, info_worker_0, info_worker_1, info_worker_2, info_worker_3, info_boss, self.company_money, self.worker_0_money, self.worker_1_money, self.worker_2_money, self.worker_3_money, self.worker_0_pay, self.worker_1_pay, self.worker_2_pay, self.worker_3_pay, self.total_days, self.worker_0_endurance, self.worker_1_endurance, self.worker_2_endurance, self.worker_3_endurance, self.worker_0_day_on, self.worker_1_day_on, self.worker_2_day_on, self.worker_3_day_on
    
    def workers_step(self, action_boss, action_worker_0, action_worker_1, action_worker_2, action_worker_3, company_money, worker_0_endurance, worker_1_endurance, worker_2_endurance, worker_3_endurance, worker_0_skill, worker_1_skill, worker_2_skill, worker_3_skill):
        if action_boss != 0:
            self.take_action_worker_0(action_worker_0, self.company_money, self.worker_0_endurance, self.worker_0_skill, self.worker_0_pay_motivation)
        if action_boss != 1:
            self.take_action_worker_1(action_worker_1, self.company_money, self.worker_1_endurance, self.worker_1_skill, self.worker_1_pay_motivation)
        if action_boss != 2:
            self.take_action_worker_2(action_worker_2, self.company_money, self.worker_2_endurance, self.worker_2_skill, self.worker_2_pay_motivation)
        if action_boss != 3:
            self.take_action_worker_3(action_worker_3, self.company_money, self.worker_3_endurance, self.worker_3_skill, self.worker_3_pay_motivation)
        return self.company_money

    def take_action_worker_0(self, action_worker_0, company_money, worker_0_endurance, worker_0_skill, worker_0_pay_motivation):
        if action_worker_0 == 0:
            if (1 - worker_0_endurance) >= 0:
                self.company_money += worker_0_skill + worker_0_pay_motivation - worker_0_endurance
        if action_worker_0 == 1:
            self.company_money += 0
        return self.company_money

    def take_action_worker_1(self, action_worker_1, company_money, worker_1_endurance, worker_1_skill, worker_1_pay_motivation):
        if action_worker_1 == 0:
            if (1 - worker_1_endurance) >= 0:
                self.company_money += worker_1_skill + worker_1_pay_motivation - worker_1_endurance
        if action_worker_1 == 1:
            self.company_money += 0
        return self.company_money

    def take_action_worker_2(self, action_worker_2, company_money, worker_2_endurance, worker_2_skill, worker_2_pay_motivation):
        if action_worker_2 == 0:
            if (1 - worker_2_endurance) >= 0:
                self.company_money += worker_2_skill + worker_2_pay_motivation - worker_2_endurance
        if action_worker_2 == 1:
            self.company_money += 0
        return self.company_money

    def take_action_worker_3(self, action_worker_3, company_money, worker_3_endurance, worker_3_skill, worker_3_pay_motivation):
        if action_worker_3 == 0:
            if (1 - worker_3_endurance) >= 0:
                self.company_money += worker_3_skill + worker_3_pay_motivation - worker_3_endurance
        if action_worker_3 == 1:
            self.company_money += 0
        return self.company_money

    def take_action_boss(self, action_boss, worker_0_pay, worker_1_pay, worker_2_pay, worker_3_pay, company_money):
        #first actions are define in step which is worker day offs
        if action_boss == 4:
            self.pay_change_0_boss_action(-2, self.worker_0_pay)
        if action_boss == 5:
            self.pay_change_0_boss_action(-1, self.worker_0_pay)
        if action_boss == 6:
            self.pay_change_0_boss_action(1, self.worker_0_pay)
        if action_boss == 7:
            self.pay_change_0_boss_action(2, self.worker_0_pay)
        if action_boss == 8:
            self.pay_change_1_boss_action(-2, self.worker_1_pay)
        if action_boss == 9:
            self.pay_change_1_boss_action(-1, self.worker_1_pay)
        if action_boss == 10:
            self.pay_change_1_boss_action(1, self.worker_1_pay)
        if action_boss == 11:
            self.pay_change_1_boss_action(2, self.worker_1_pay)
        if action_boss == 12:
            self.pay_change_2_boss_action(-2, self.worker_2_pay)
        if action_boss == 13:
            self.pay_change_2_boss_action(-1, self.worker_2_pay)
        if action_boss == 14:
            self.pay_change_2_boss_action(1, self.worker_2_pay)
        if action_boss == 15:
            self.pay_change_2_boss_action(2, self.worker_2_pay)
        if action_boss == 16:
            self.pay_change_3_boss_action(-2, self.worker_3_pay)
        if action_boss == 17:
            self.pay_change_3_boss_action(-1, self.worker_3_pay)
        if action_boss == 18:
            self.pay_change_3_boss_action(1, self.worker_3_pay)
        if action_boss == 19:
            self.pay_change_3_boss_action(2, self.worker_3_pay)
        if action_boss == 20:
            company_money += 0
        if action_boss == 21:
            company_money += 20
        return self.company_money, self.worker_0_pay, self.worker_1_pay

    def pay_change_0_boss_action(self, pay_change, worker_0_pay):
        if (self.worker_0_pay + pay_change) >= 1:
            self.worker_0_pay += pay_change
        return self.worker_0_pay
        
    def pay_change_1_boss_action(self, pay_change, worker_1_pay):
        if (self.worker_1_pay + pay_change) >= 1:
            self.worker_1_pay += pay_change
        return self.worker_1_pay

    def pay_change_2_boss_action(self, pay_change, worker_2_pay):
        if (self.worker_2_pay + pay_change) >= 1:
            self.worker_2_pay += pay_change
        return self.worker_2_pay
        
    def pay_change_3_boss_action(self, pay_change, worker_3_pay):
        if (self.worker_3_pay + pay_change) >= 1:
            self.worker_3_pay += pay_change
        return self.worker_3_pay

    def reset(self):
        self.state_boss = 0
        self.company_money = 100
        self.total_days = 365

        self.state_worker_0 = 0 
        self.state_worker_1 = 0  
        self.state_worker_2 = 0 
        self.state_worker_3 = 0
        self.worker_0_pay = 1
        self.worker_1_pay = 1
        self.worker_2_pay = 1
        self.worker_3_pay = 1
        self.worker_0_money = 0
        self.worker_1_money = 0
        self.worker_2_money = 0
        self.worker_3_money = 0
        self.worker_0_day_on = 0
        self.worker_1_day_on = 0
        self.worker_2_day_on = 0
        self.worker_3_day_on = 0
        self.worker_0_endurance = 0
        self.worker_1_endurance = 0  
        self.worker_2_endurance = 0
        self.worker_3_endurance = 0  
        self.worker_0_pay_motivation = 0
        self.worker_1_pay_motivation = 0
        self.worker_2_pay_motivation = 0
        self.worker_3_pay_motivation = 0

        return self.state_worker_0, self.state_worker_1, self.state_worker_2, self.state_worker_3, self.worker_0_pay, self.worker_1_pay, self.worker_2_pay, self.worker_3_pay, self.worker_0_money, self.worker_1_money, self.worker_2_money, self.worker_3_money, self.worker_0_day_on, self.worker_1_day_on, self.worker_2_day_on, self.worker_3_day_on, self.worker_0_endurance, self.worker_1_endurance,  self.worker_2_endurance, self.worker_3_endurance,  self.worker_0_pay_motivation, self.worker_1_pay_motivation, self.worker_2_pay_motivation, self.worker_3_pay_motivation,
        

#**DQN**

In [ ]:
class boss_DQN:
    def __init__(self, action_space_boss, state_boss):
        self.action_space = 22
        self.state_space = 1
        self.lr = 0.001
        self.epsilon_decay = 0.998
        self.batch_size = 50
        self.epsilon_min = 0.02
        self.epsilon = 8.1
        self.gamma = 0.99
        self.memory = deque(maxlen = 1000000)
        self.model = self.boss_model()

    def boss_model(self):
        model = Sequential()
        model.add(Dense(1, input_dim = (1), activation=relu))
        model.add(Dense(150, activation=relu))
        model.add(Dense(120, activation=relu))
        model.add(Dense(self.action_space, activation=linear))
        model.compile(loss="mse", optimizer=Adam(lr=self.lr))
        model.summary()
        return model
 
    def new_memory(self, state_boss, boss_action, boss_reward, next_state, done_state):
        self.memory.append((state_boss, boss_action, boss_reward, next_state, done_state))
  
    def predict(self, state_boss):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.action_space, 1) 

        predict_values = (self.model.predict([self.state_space])) 
        return np.argmax(predict_values[0])
  
    def replay(self): 
 
        if len(self.memory) < self.batch_size:
            return
            
        sample = random.sample(self.memory, self.batch_size) 
 
        states_boss = np.array([i[0] for i in sample]) 
        actions_boss = np.array([i[1] for i in sample]) 
        rewards_boss = np.array([i[2] for i in sample])
        next_states_boss = np.array([i[3] for i in sample]) 
        dones_boss = np.array([i[4] for i in sample])

        Qtargets = rewards_boss + self.gamma*(np.amax(self.model.predict_on_batch(next_states_boss), axis=1))*(1-dones_boss) 
        Qtarget = self.model.predict_on_batch(states_boss) 
        batch_size_array = np.array([i for i in range(self.batch_size)]) 

        self.model.fit(states_boss, Qtarget, epochs=1, verbose=0) 
        if self.epsilon > self.epsilon_min: 
            self.epsilon *= self.epsilon_decay



#**Training**

In [ ]:
env = company_env()
episodes = 100

score_boss_list = []
score_worker_0_list = []
worker_0_endurance_list = []
worker_0_day_on_list = []
worker_0_pay_list = []
worker_1_endurance_list = []
worker_1_day_on_list = []
worker_1_pay_list = []
score_worker_1_list = []
score_worker_2_list = []
worker_2_endurance_list = []
worker_2_day_on_list = []
worker_2_pay_list = []
worker_3_endurance_list = []
worker_3_day_on_list = []
worker_3_pay_list = []
score_worker_3_list = []

boss_agent = boss_DQN(env.action_space_boss, env.state_boss)

for episode in range(1, episodes+1):
    env.reset()
    done = False

    score_boss = 0
    score_worker_0 = 0
    score_worker_1 = 0
    score_worker_2 = 0
    score_worker_3 = 0
    temp_memory = 0

    while done == False:
        action_boss = boss_agent.predict(env.state_boss)
        env.worker_0_pay_motivation, env.worker_1_pay_motivation, env.worker_2_pay_motivation, env.worker_3_pay_motivation, env.state_worker_0, env.state_worker_1, env.state_worker_2, env.state_worker_3, env.state_boss, env.boss_reward, env.worker_0_reward, env.worker_1_reward, env.worker_2_reward, env.worker_3_reward, done, info_worker_0, info_worker_1, info_worker_2, info_worker_3, info_boss, env.company_money, env.worker_0_money, env.worker_1_money, env.worker_2_money, env.worker_3_money, env.worker_0_pay, env.worker_1_pay, env.worker_2_pay, env.worker_3_pay, env.total_days, env.worker_0_endurance, env.worker_1_endurance, env.worker_2_endurance, env.worker_3_endurance, env.worker_0_day_on, env.worker_1_day_on, env.worker_2_day_on, env.worker_3_day_on = env.boss_step(action_boss)

        for i in range(0,481):
            action_worker_0 = np.random.choice(env.action_space_worker_0, 1)
            action_worker_1 = np.random.choice(env.action_space_worker_1, 1)
            action_worker_2 = np.random.choice(env.action_space_worker_2, 1)
            action_worker_3 = np.random.choice(env.action_space_worker_3, 1)
            env.workers_step(action_boss, action_worker_0, action_worker_1, action_worker_2, action_worker_3, env.company_money, env.worker_0_endurance, env.worker_1_endurance, env.worker_2_endurance, env.worker_3_endurance, env.worker_0_skill, env.worker_1_skill, env.worker_2_skill, env.worker_3_skill)

        boss_next_state = temp_memory
        temp_memory = env.state_boss

        boss_agent.new_memory(env.state_boss, action_boss, env.boss_reward, boss_next_state, done)
        boss_agent.replay()

    worker_0_pay_list.append(env.worker_0_pay)
    worker_1_pay_list.append(env.worker_1_pay)
    worker_2_pay_list.append(env.worker_2_pay)
    worker_3_pay_list.append(env.worker_3_pay)
    worker_0_day_on_list.append(env.worker_0_day_on)
    worker_1_day_on_list.append(env.worker_1_day_on)
    worker_2_day_on_list.append(env.worker_2_day_on)
    worker_3_day_on_list.append(env.worker_3_day_on)
    worker_0_endurance_list.append(env.worker_0_endurance)
    worker_1_endurance_list.append(env.worker_1_endurance)
    worker_2_endurance_list.append(env.worker_2_endurance)
    worker_3_endurance_list.append(env.worker_3_endurance)

    score_boss = env.boss_reward
    score_worker_0 = env.worker_0_reward
    score_worker_1 = env.worker_1_reward
    score_worker_2 = env.worker_2_reward
    score_worker_3 = env.worker_3_reward
    score_worker_0_list.append(score_worker_0)
    score_worker_1_list.append(score_worker_1)
    score_worker_2_list.append(score_worker_2)
    score_worker_3_list.append(score_worker_3)
    
    score_boss_list.append(score_boss)

    print(f"Episode:{episode} Score Boss: {score_boss} Score Worker Zero: {score_worker_0} Score Worker One: {score_worker_1} Score Worker Two: {score_worker_2} Score Worker Three: {score_worker_3}")

    avg_score_boss_list = np.mean(score_boss_list[-100:]) 
    avg_score_worker_0_list = np.mean(score_worker_0_list[-100:]) 
    avg_score_worker_1_list = np.mean(score_worker_1_list[-100:]) 
    avg_score_worker_2_list = np.mean(score_worker_2_list[-100:]) 
    avg_score_worker_3_list = np.mean(score_worker_3_list[-100:]) 

    if episode > 99:
        print(f"Average Score Over the Past 100 Episode for Boss: {avg_score_boss_list}, Worker Zero: {avg_score_worker_0_list}, Worker One: {avg_score_worker_1_list}, Worker Two: {avg_score_worker_2_list}, Worker Three: {avg_score_worker_3_list}") 
    else:
        print(f"Average Score Over the Past {episode} Episode for Boss: {avg_score_boss_list}, Worker Zero: {avg_score_worker_0_list}, Worker One: {avg_score_worker_1_list}, Worker Two: {avg_score_worker_2_list}, Worker Three: {avg_score_worker_3_list}")
    print()

#boss score over time
fig, boss_score_over_time = plt.subplots()
boss_score_over_time.plot([i+1 for i in range(0, len(score_boss_list), 2)], score_boss_list[::2])
boss_score_over_time.set(xlabel="Time (episode)", ylabel="Boss Score (company_money)", title="Boss Score over Time")



#worker 0 score over time
fig, worker_0_score_over_time = plt.subplots()
worker_0_score_over_time.plot([i+1 for i in range(0, len(score_worker_0_list), 2)], score_worker_0_list[::2])
worker_0_score_over_time.set(xlabel="Time (episode)", ylabel="Worker Zero Score (worker_0_money)", title="Worker Zero Score over Time")

#worker 1 score over time
fig, worker_1_score_over_time = plt.subplots()
worker_1_score_over_time.plot([i+1 for i in range(0, len(score_worker_1_list), 2)], score_worker_1_list[::2])
worker_1_score_over_time.set(xlabel="Time (episode)", ylabel="Worker One Score (worker_1_money)", title="Worker One Score over Time")

#worker 2 score over time
fig, worker_2_score_over_time = plt.subplots()
worker_2_score_over_time.plot([i+1 for i in range(0, len(score_worker_2_list), 2)], score_worker_2_list[::2])
worker_2_score_over_time.set(xlabel="Time (episode)", ylabel="Worker Two Score (worker_2_money)", title="Worker Two Score over Time")

#worker 3 score over time
fig, worker_3_score_over_time = plt.subplots()
worker_3_score_over_time.plot([i+1 for i in range(0, len(score_worker_3_list), 2)], score_worker_3_list[::2])
worker_3_score_over_time.set(xlabel="Time (episode)", ylabel="Worker Three Score (worker_3_money)", title="Worker Three Score over Time")


#worker 0 endurance over time
fig, worker_0_endurance_over_time = plt.subplots()
worker_0_endurance_over_time.plot([i+1 for i in range(0, len(worker_0_endurance_list), 2)], worker_0_endurance_list[::2])
worker_0_endurance_over_time.set(xlabel="Time (episode)", ylabel="Worker Zero Endurance (Worker_0_Endurance)", title="Worker Zero Endurance over Time")

#worker 1 endurance over time
fig, worker_1_endurance_over_time = plt.subplots()
worker_1_endurance_over_time.plot([i+1 for i in range(0, len(worker_1_endurance_list), 2)], worker_1_endurance_list[::2])
worker_1_endurance_over_time.set(xlabel="Time (episode)", ylabel="Worker One Endurance (Worker_1_Endurance)", title="Worker One Endurance over Time")

#worker 2 endurance over time
fig, worker_2_endurance_over_time = plt.subplots()
worker_2_endurance_over_time.plot([i+1 for i in range(0, len(worker_2_endurance_list), 2)], worker_2_endurance_list[::2])
worker_2_endurance_over_time.set(xlabel="Time (episode)", ylabel="Worker Two Endurance (Worker_2_Endurance)", title="Worker Two Endurance over Time")

#worker 3 endurance over time
fig, worker_3_endurance_over_time = plt.subplots()
worker_3_endurance_over_time.plot([i+1 for i in range(0, len(worker_3_endurance_list), 2)], worker_3_endurance_list[::2])
worker_3_endurance_over_time.set(xlabel="Time (episode)", ylabel="Worker Three Endurance (Worker_3_Endurance)", title="Worker Three Endurance over Time")


#Worker 0 Day On
fig, worker_0_day_on_over_time = plt.subplots()
worker_0_day_on_over_time.plot([i+1 for i in range(0, len(worker_0_day_on_list), 2)], worker_0_day_on_list[::2])
worker_0_day_on_over_time.set(xlabel="Time (episode)", ylabel="Worker Zero Days Working (worker_0_day_on)", title="Worker Zero Days Working over Time")

#Worker 1 Day On
fig, worker_1_day_on_over_time = plt.subplots()
worker_1_day_on_over_time.plot([i+1 for i in range(0, len(worker_1_day_on_list), 2)], worker_1_day_on_list[::2])
worker_1_day_on_over_time.set(xlabel="Time (episode)", ylabel="Worker One Days Working (worker_1_day_on)", title="Worker One Days Working over Time")

#Worker 2 Day On
fig, worker_2_day_on_over_time = plt.subplots()
worker_2_day_on_over_time.plot([i+1 for i in range(0, len(worker_2_day_on_list), 2)], worker_2_day_on_list[::2])
worker_2_day_on_over_time.set(xlabel="Time (episode)", ylabel="Worker Two Days Working (worker_2_day_on)", title="Worker Two Days Working over Time")

#Worker 3 Day On
fig, worker_3_day_on_over_time = plt.subplots()
worker_3_day_on_over_time.plot([i+1 for i in range(0, len(worker_3_day_on_list), 2)], worker_3_day_on_list[::2])
worker_3_day_on_over_time.set(xlabel="Time (episode)", ylabel="Worker Three Days Working (worker_3_day_on)", title="Worker Three Days Working over Time")


#Worker 0 Pay
fig, worker_0_pay_over_time = plt.subplots()
worker_0_pay_over_time.plot([i+1 for i in range(0, len(worker_0_pay_list), 2)], worker_0_pay_list[::2])
worker_0_pay_over_time.set(xlabel="Time (episode)", ylabel="Worker Zero Pay (worker_0_pay)", title="Worker Zero Pay over Time")

#Worker 0 Pay
fig, worker_1_pay_over_time = plt.subplots()
worker_1_pay_over_time.plot([i+1 for i in range(0, len(worker_1_pay_list), 2)], worker_1_pay_list[::2])
worker_1_pay_over_time.set(xlabel="Time (episode)", ylabel="Worker One Pay (worker_1_pay)", title="Worker One Pay over Time")

#Worker 2 Pay
fig, worker_2_pay_over_time = plt.subplots()
worker_2_pay_over_time.plot([i+1 for i in range(0, len(worker_2_pay_list), 2)], worker_2_pay_list[::2])
worker_2_pay_over_time.set(xlabel="Time (episode)", ylabel="Worker Two Pay (worker_2_pay)", title="Worker Two Pay over Time")

#Worker 3 Pay
fig, worker_3_pay_over_time = plt.subplots()
worker_3_pay_over_time.plot([i+1 for i in range(0, len(worker_3_pay_list), 2)], worker_3_pay_list[::2])
worker_3_pay_over_time.set(xlabel="Time (episode)", ylabel="Worker Three Pay (worker_3_pay)", title="Worker Three Pay over Time")

plt.show()
